In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution, kl_reg

# Data

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 18:13:59.468006 140459681621824 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 18:13:59.475238 140459681621824 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 18:13:59.480487 140459681621824 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 18:13:59.488501 140459681621824 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 18:13:59.493021 140459681621824 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 18:13:59.500934 140459681621824 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 18:13:59.545617 140459681621824 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]                  
                                                                 input_1[0][0]                    
__________

In [13]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 18:13:59.625610 140459681621824 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9280 train_acc= 0.2929 val_loss= 1.9276 val_acc= 0.3500 time= 1.1030
Epoch: 0002 train_loss= 1.9084 train_acc= 0.2929 val_loss= 1.9086 val_acc= 0.3500 time= 0.0174
Epoch: 0003 train_loss= 1.8874 train_acc= 0.2929 val_loss= 1.8887 val_acc= 0.3500 time= 0.0181
Epoch: 0004 train_loss= 1.8652 train_acc= 0.2929 val_loss= 1.8678 val_acc= 0.3500 time= 0.0236
Epoch: 0005 train_loss= 1.8421 train_acc= 0.2929 val_loss= 1.8461 val_acc= 0.3500 time= 0.0174
Epoch: 0006 train_loss= 1.8201 train_acc= 0.2929 val_loss= 1.8257 val_acc= 0.3500 time= 0.0177
Epoch: 0007 train_loss= 1.7993 train_acc= 0.2929 val_loss= 1.8067 val_acc= 0.3500 time= 0.0191
Epoch: 0008 train_loss= 1.7798 train_acc= 0.2929 val_loss= 1.7896 val_acc= 0.3500 time= 0.0231
Epoch: 0009 train_loss= 1.7620 train_acc= 0.2929 val_loss= 1.7746 val_acc= 0.3500 time= 0.0200
Epoch: 0010 train_loss= 1.7460 train_acc= 0.2929 val_loss= 1.7621 val_acc= 0.3500 time= 0.0188
Epoch: 0011 train_loss= 1.7314 train_acc= 0.2929 v

Epoch: 0094 train_loss= 0.7293 train_acc= 0.9071 val_loss= 1.0332 val_acc= 0.7733 time= 0.0180
Epoch: 0095 train_loss= 0.7221 train_acc= 0.9071 val_loss= 1.0279 val_acc= 0.7767 time= 0.0184
Epoch: 0096 train_loss= 0.7148 train_acc= 0.9071 val_loss= 1.0215 val_acc= 0.7767 time= 0.0183
Epoch: 0097 train_loss= 0.7078 train_acc= 0.9071 val_loss= 1.0146 val_acc= 0.7800 time= 0.0179
Epoch: 0098 train_loss= 0.7013 train_acc= 0.9071 val_loss= 1.0078 val_acc= 0.7800 time= 0.0185
Epoch: 0099 train_loss= 0.6949 train_acc= 0.9071 val_loss= 1.0011 val_acc= 0.7767 time= 0.0190
Epoch: 0100 train_loss= 0.6884 train_acc= 0.9071 val_loss= 0.9946 val_acc= 0.7733 time= 0.0183
Epoch: 0101 train_loss= 0.6821 train_acc= 0.9071 val_loss= 0.9888 val_acc= 0.7733 time= 0.0189
Epoch: 0102 train_loss= 0.6754 train_acc= 0.9071 val_loss= 0.9835 val_acc= 0.7800 time= 0.0173
Epoch: 0103 train_loss= 0.6682 train_acc= 0.9143 val_loss= 0.9794 val_acc= 0.7767 time= 0.0240
Epoch: 0104 train_loss= 0.6617 train_acc= 0.9286 v

Epoch: 0181 train_loss= 0.3985 train_acc= 0.9786 val_loss= 0.7860 val_acc= 0.8133 time= 0.0163
Epoch: 0182 train_loss= 0.3960 train_acc= 0.9786 val_loss= 0.7833 val_acc= 0.8133 time= 0.0185
Epoch: 0183 train_loss= 0.3934 train_acc= 0.9786 val_loss= 0.7784 val_acc= 0.8133 time= 0.0169
Epoch: 0184 train_loss= 0.3925 train_acc= 0.9643 val_loss= 0.7757 val_acc= 0.8133 time= 0.0171
Epoch: 0185 train_loss= 0.3914 train_acc= 0.9643 val_loss= 0.7747 val_acc= 0.8067 time= 0.0236
Epoch: 0186 train_loss= 0.3888 train_acc= 0.9643 val_loss= 0.7747 val_acc= 0.8033 time= 0.0186
Epoch: 0187 train_loss= 0.3857 train_acc= 0.9643 val_loss= 0.7730 val_acc= 0.7967 time= 0.0194
Epoch: 0188 train_loss= 0.3832 train_acc= 0.9643 val_loss= 0.7706 val_acc= 0.7967 time= 0.0282
Epoch: 0189 train_loss= 0.3808 train_acc= 0.9714 val_loss= 0.7672 val_acc= 0.8033 time= 0.0190
Epoch: 0190 train_loss= 0.3792 train_acc= 0.9786 val_loss= 0.7673 val_acc= 0.8033 time= 0.0187
Epoch: 0191 train_loss= 0.3781 train_acc= 0.9786 v

In [14]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.7945
accuracy = 0.8170


# Gaussian Graph Convolution

In [15]:
B1 = 5e-4
B2 = 5e-4

In [16]:
H = Dropout(0.5)(X_in)
H1 = GaussianGraphConvolution(16,
    is_first=True,
    activation='relu',
    mean_regularizer=l2(B1),
    variance_regularizer=l2(B1),
#     dropout=0.5
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    activation='softmax',
)(H1+G)

W1110 18:14:04.660465 140459681621824 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/ggcl.py:165: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W1110 18:14:04.661660 140459681621824 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/distributions/normal.py:160: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should updat

In [17]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
model.add_loss(kl_reg(*H1, B2), H1)

W1110 18:14:04.694933 140459681621824 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/losses.py:28: kl_divergence (from tensorflow.python.ops.distributions.kullback_leibler) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [18]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
gaussian_graph_convolution_1 (G [(None, 16), (None,  45856       dropout_3[0][0]                  
                                                                 input_1[0][0]                    
__________

In [19]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

Epoch: 0001 train_loss= 1.9385 train_acc= 0.3071 val_loss= 1.9411 val_acc= 0.2933 time= 0.4902
Epoch: 0002 train_loss= 1.9291 train_acc= 0.4143 val_loss= 1.9347 val_acc= 0.3133 time= 0.0232
Epoch: 0003 train_loss= 1.9227 train_acc= 0.3786 val_loss= 1.9275 val_acc= 0.3300 time= 0.0198
Epoch: 0004 train_loss= 1.9083 train_acc= 0.4143 val_loss= 1.9195 val_acc= 0.3300 time= 0.0189
Epoch: 0005 train_loss= 1.8978 train_acc= 0.4214 val_loss= 1.9107 val_acc= 0.3700 time= 0.0240
Epoch: 0006 train_loss= 1.8850 train_acc= 0.4071 val_loss= 1.9025 val_acc= 0.3400 time= 0.0195
Epoch: 0007 train_loss= 1.8714 train_acc= 0.4500 val_loss= 1.8919 val_acc= 0.3667 time= 0.0247
Epoch: 0008 train_loss= 1.8590 train_acc= 0.4714 val_loss= 1.8831 val_acc= 0.3600 time= 0.0233
Epoch: 0009 train_loss= 1.8443 train_acc= 0.4929 val_loss= 1.8735 val_acc= 0.3867 time= 0.0192
Epoch: 0010 train_loss= 1.8305 train_acc= 0.4857 val_loss= 1.8625 val_acc= 0.3867 time= 0.0195
Epoch: 0011 train_loss= 1.8153 train_acc= 0.4857 v

Epoch: 0093 train_loss= 0.8308 train_acc= 0.8786 val_loss= 1.0885 val_acc= 0.7233 time= 0.0232
Epoch: 0094 train_loss= 0.8244 train_acc= 0.8714 val_loss= 1.0828 val_acc= 0.7300 time= 0.0203
Epoch: 0095 train_loss= 0.8172 train_acc= 0.8786 val_loss= 1.0777 val_acc= 0.7333 time= 0.0188
Epoch: 0096 train_loss= 0.8097 train_acc= 0.8786 val_loss= 1.0723 val_acc= 0.7300 time= 0.0231
Epoch: 0097 train_loss= 0.8041 train_acc= 0.8786 val_loss= 1.0665 val_acc= 0.7333 time= 0.0240
Epoch: 0098 train_loss= 0.7978 train_acc= 0.8857 val_loss= 1.0615 val_acc= 0.7400 time= 0.0240
Epoch: 0099 train_loss= 0.7907 train_acc= 0.8857 val_loss= 1.0566 val_acc= 0.7500 time= 0.0290
Epoch: 0100 train_loss= 0.7846 train_acc= 0.8857 val_loss= 1.0514 val_acc= 0.7500 time= 0.0234
Epoch: 0101 train_loss= 0.7784 train_acc= 0.8857 val_loss= 1.0464 val_acc= 0.7533 time= 0.0246
Epoch: 0102 train_loss= 0.7719 train_acc= 0.8857 val_loss= 1.0410 val_acc= 0.7533 time= 0.0287
Epoch: 0103 train_loss= 0.7655 train_acc= 0.8857 v

Epoch: 0188 train_loss= 0.4286 train_acc= 0.9714 val_loss= 0.7831 val_acc= 0.8300 time= 0.0232
Epoch: 0189 train_loss= 0.4258 train_acc= 0.9714 val_loss= 0.7814 val_acc= 0.8300 time= 0.0251
Epoch: 0190 train_loss= 0.4236 train_acc= 0.9714 val_loss= 0.7797 val_acc= 0.8300 time= 0.0257
Epoch: 0191 train_loss= 0.4216 train_acc= 0.9714 val_loss= 0.7779 val_acc= 0.8300 time= 0.0251
Epoch: 0192 train_loss= 0.4195 train_acc= 0.9714 val_loss= 0.7761 val_acc= 0.8300 time= 0.0268
Epoch: 0193 train_loss= 0.4172 train_acc= 0.9714 val_loss= 0.7743 val_acc= 0.8300 time= 0.0203
Epoch: 0194 train_loss= 0.4152 train_acc= 0.9714 val_loss= 0.7726 val_acc= 0.8300 time= 0.0207
Epoch: 0195 train_loss= 0.4132 train_acc= 0.9714 val_loss= 0.7710 val_acc= 0.8300 time= 0.0224
Epoch: 0196 train_loss= 0.4103 train_acc= 0.9714 val_loss= 0.7699 val_acc= 0.8300 time= 0.0284
Epoch: 0197 train_loss= 0.4085 train_acc= 0.9714 val_loss= 0.7688 val_acc= 0.8300 time= 0.0200
Epoch: 0198 train_loss= 0.4068 train_acc= 0.9714 v

In [20]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.8279
accuracy = 0.8290


# Attacks